In [ ]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, adjusted_rand_score, normalized_mutual_info_score
import matplotlib.pyplot as plt

# Classe não indexada / Class not indexed

In [ ]:
# ===== 1. Ler os dados .XLSX / Read .XLSX data =====
dados = pd.read_excel("NAME.xlsx")
print(dados.head())

In [ ]:
# === Ler os dados .CSV / Read .CSV data ===
dados = pd.read_csv('NAME.csv', sep=';')  # especifica o separador / specifies the separator
print(dados.head())

In [ ]:
# === Padronizar os dados / Standardize data ===
scaler = StandardScaler()
dados_padronizados = scaler.fit_transform(dados)

In [ ]:
# === Calcular matriz de distâncias Euclidiana / Calculate Euclidean distance matrix ===
distancia = pdist(dados_padronizados, metric='euclidean')
matriz_de_distancias = squareform(distancia)

print("Distance Matrix:\n", matriz_de_distancias)

# Salvar matriz de distâncias / Save distance matrix
pd.DataFrame(matriz_de_distancias).to_excel("matriz_de_distancias.xlsx", index=False)
pd.DataFrame(matriz_de_distancias).to_csv("matriz_de_distancias.csv", index=False)

In [ ]:
# === Clustering hierárquico (ligação simples) / Hierarchical clustering (single linkage) ===
agrupamento = linkage(distancia, method='single')

In [ ]:
# === Criar tabela de amalgamação / Create amalgamation table ===
n_obs = dados.shape[0]
amalgamation_steps = agrupamento[:, :2].astype(int)  # pares de clusters unidos / pairs of joined clusters
distancias = agrupamento[:, 2]

amalgamation_table = pd.DataFrame({
    "Step": np.arange(1, len(agrupamento) + 1),
    "Number_of_Clusters": n_obs - np.arange(1, len(agrupamento) + 1),
    "Similarity_Level": None,
    "Distance_Level": distancias,
    "Clusters_joined": None,
    "New_cluster": -np.arange(1, len(agrupamento) + 1),
    "Number_of_observations_in_new_cluster": None
})

# Preencher colunas adicionais / Fill in additional columns
max_distance = max(distancias)

for i in range(len(agrupamento)):
    cluster1, cluster2 = int(agrupamento[i, 0]), int(agrupamento[i, 1])
    amalgamation_table.loc[i, "Clusters_joined"] = f"{cluster1} & {cluster2}"
    # Fórmula de similaridade / Similarity formula
    amalgamation_table.loc[i, "Similarity_Level"] = 100 * (1 - distancias[i] / max_distance)
    # Número de observações acumuladas no novo cluster / Number of observations accumulated in the new cluster
    count = np.sum(agrupamento[:i+1, :2] < 0) + (i + 1)
    amalgamation_table.loc[i, "Number_of_observations_in_new_cluster"] = count

print(amalgamation_table)

# Salvar tabela / Save table
amalgamation_table.to_excel("amalgamation_table.xlsx", index=False)
amalgamation_table.to_csv("amalgamation_table.csv", index=False)

In [ ]:
# === Gráfico: Similaridade e Distância / Graph: Similarity and Distance ===
fig, ax1 = plt.subplots()

ax1.plot(amalgamation_table["Step"], amalgamation_table["Similarity_Level"], label="Similaridade", color="blue")
ax1.set_xlabel("Etapa")
ax1.set_ylabel("Nível de Similaridade", color="blue")
ax1.tick_params(axis='y', labelcolor="blue")

ax2 = ax1.twinx()
ax2.plot(amalgamation_table["Step"], amalgamation_table["Distance_Level"], label="Distância", color="red")
ax2.set_ylabel("Nível de Distância", color="red")
ax2.tick_params(axis='y', labelcolor="red")

plt.title("Screeplot: etapas no agrupamento hierárquico - observações")
plt.show()

In [ ]:
# === Dendrograma / Dendrogram ===
plt.figure(figsize=(12, 6))
dendrogram(agrupamento)
plt.title("Dendrogram (Simple Link - Euclidean Distance - observations)")
plt.xlabel("Observations")
plt.ylabel("Distance")
plt.show()

In [ ]:
# === Exemplo: cortar dendrograma em k clusters / Example: cutting dendrogram into k clusters ===
k = 2  # pode ajustar esse número / you can adjust this number
clusters = fcluster(agrupamento, k, criterion='maxclust')


In [ ]:
# === PCA para projeção em 2D / PCA for 2D projection ===
pca = PCA(n_components=2)
dados_pca = pca.fit_transform(dados_padronizados)

# === Scatterplot dos clusters ===
plt.figure(figsize=(10, 6))
scatter = plt.scatter(dados_pca[:, 0], dados_pca[:, 1], c=clusters, cmap="tab10", s=50, alpha=0.8)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title(f"Hierarchical Grouping ({k} clusters) - PCA 2D")
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.show()

In [ ]:
# Adicionar rótulos dos clusters ao dataframe original / Add cluster labels to the original dataframe
dados["Cluster"] = clusters
# Mover a coluna 'Cluster' para a primeira posição / Move the 'Cluster' column to the first position
cluster_column = dados.pop("Cluster")
dados.insert(0, "Cluster", cluster_column)


dados.to_excel("dados_com_clusters.xlsx", index=False)
dados.to_csv("dados_com_clusters.csv", index=False)
print(dados.head())

# Classe indexada / Indexed class

In [ ]:
# ===== 1. Ler os dados .XLSX / Read .XLSX data =====
dados = pd.read_excel("NAME.xlsx")
print(dados.head())

In [ ]:
# === Ler os dados .CSV / Read .CSV data ===
dados = pd.read_csv('NAME.csv', sep=',')  # especifica o separador / specifies the separator
print(dados.head())

In [ ]:
# Separar classes (1ª coluna) e variáveis numéricas / Separate classes (1st column) and numeric variables
classes = dados.iloc[:, 0]   # rótulos das classes / class labels
dados_num = dados.iloc[:, 1:]  # apenas colunas numéricas / only numeric columns

# === Padronizar os dados numéricos / Standardize numerical data ===
scaler = StandardScaler()
dados_padronizados = scaler.fit_transform(dados_num)

In [ ]:
# === Calcular matriz de distâncias Euclidiana / Calculate Euclidean distance matrix ===
distancia = pdist(dados_padronizados, metric='euclidean')
matriz_de_distancias = squareform(distancia)

print("Distance Matrix:\n", matriz_de_distancias)

# Salvar matriz de distâncias / Save distance matrix
pd.DataFrame(matriz_de_distancias).to_excel("matriz_de_distancias.xlsx", index=False)
pd.DataFrame(matriz_de_distancias).to_csv("matriz_de_distancias.csv", index=False)

In [ ]:
# === Clustering hierárquico (ligação simples) / Hierarchical clustering (single linkage) ===
agrupamento = linkage(distancia, method='single')

# === Criar tabela de amalgamação / Create amalgamation table ===
n_obs = dados.shape[0]
amalgamation_steps = agrupamento[:, :2].astype(int)
distancias = agrupamento[:, 2]

amalgamation_table = pd.DataFrame({
    "Step": np.arange(1, len(agrupamento) + 1),
    "Number_of_Clusters": n_obs - np.arange(1, len(agrupamento) + 1),
    "Similarity_Level": None,
    "Distance_Level": distancias,
    "Clusters_joined": None,
    "New_cluster": -np.arange(1, len(agrupamento) + 1),
    "Number_of_observations_in_new_cluster": None
})

# Preencher colunas adicionais / Fill in additional columns
max_distance = max(distancias)
for i in range(len(agrupamento)):
    cluster1, cluster2 = int(agrupamento[i, 0]), int(agrupamento[i, 1])
    amalgamation_table.loc[i, "Clusters_joined"] = f"{cluster1} & {cluster2}"
    amalgamation_table.loc[i, "Similarity_Level"] = 100 * (1 - distancias[i] / max_distance)
    count = np.sum(agrupamento[:i+1, :2] < 0) + (i + 1)
    amalgamation_table.loc[i, "Number_of_observations_in_new_cluster"] = count

print(amalgamation_table)
amalgamation_table.to_excel("amalgamation_table.xlsx", index=False)
amalgamation_table.to_csv("amalgamation_table.csv", index=False)

In [ ]:
# === Gráfico: Similaridade e Distância / Graph: Similarity and Distance ===
fig, ax1 = plt.subplots()
ax1.plot(amalgamation_table["Step"], amalgamation_table["Similarity_Level"], label="Similaridade", color="blue")
ax1.set_xlabel("Etapa")
ax1.set_ylabel("Nível de Similaridade", color="blue")
ax1.tick_params(axis='y', labelcolor="blue")
ax2 = ax1.twinx()
ax2.plot(amalgamation_table["Step"], amalgamation_table["Distance_Level"], label="Distância", color="red")
ax2.set_ylabel("Nível de Distância", color="red")
ax2.tick_params(axis='y', labelcolor="red")
plt.title("Screeplot: etapas no agrupamento hierárquico - observações")
plt.show()

In [ ]:
# === Dendrograma / Dendrogram ===
plt.figure(figsize=(12, 6))
dendrogram(agrupamento)
plt.title("Dendrogram (Simple Link - Euclidean Distance - observations)")
plt.xlabel("Observations")
plt.ylabel("Distance")
plt.show()

In [ ]:
# === Exemplo: cortar dendrograma em k clusters / Example: cutting dendrogram into k clusters ===
k = len(np.unique(classes))  # usar o número de classes reais como referência / use the number of real classes as a reference
clusters = fcluster(agrupamento, k, criterion='maxclust')

# === PCA para projeção em 2D / PCA for 2D projection ===
pca = PCA(n_components=2)
dados_pca = pca.fit_transform(dados_padronizados)

plt.figure(figsize=(10, 6))
scatter = plt.scatter(dados_pca[:, 0], dados_pca[:, 1], c=clusters, cmap="tab10", s=50, alpha=0.8)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title(f"Hierarchical Grouping ({k} clusters) - PCA 2D")
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.show()

In [ ]:
# === Avaliar proximidade com as classes reais / Assess proximity to real classes ===
# ARI e NMI
ari = adjusted_rand_score(classes, clusters)
nmi = normalized_mutual_info_score(classes, clusters)
print(f"Adjusted Rand Index (ARI): {ari:.4f}")
print(f"Normalized Mutual Information (NMI): {nmi:.4f}")